In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Load dataset
dataset = pd.read_csv('Nasik_lstm_10.csv')
data = dataset.values
data = data.astype('float32')

In [3]:
# Normalize the data
scaler = MinMaxScaler(feature_range=(0, 1))
data = scaler.fit_transform(data)

In [4]:
# Split data into training and testing sets
train_size = int(len(data) * 0.67)
test_size = len(data) - train_size
train, test = data[0:train_size,:], data[train_size:len(data),:]

In [5]:
# Create function to create LSTM model
def create_model():
    model = Sequential()
    model.add(LSTM(50, input_shape=(1, 1)))
    model.add(Dropout(0.2))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

In [6]:
# Reshape input data to be 3-dimensional
X_train = np.reshape(train, (train.shape[0], 1, train.shape[1]))
X_test = np.reshape(test, (test.shape[0], 1, test.shape[1]))

In [7]:
# Create and fit the LSTM model
model = create_model()
model.fit(X_train, train, epochs=100, batch_size=1, verbose=2)

Epoch 1/100
1026/1026 - 6s - loss: 0.0131 - 6s/epoch - 6ms/step
Epoch 2/100
1026/1026 - 3s - loss: 0.0013 - 3s/epoch - 3ms/step
Epoch 3/100
1026/1026 - 3s - loss: 9.8484e-04 - 3s/epoch - 2ms/step
Epoch 4/100
1026/1026 - 3s - loss: 8.9632e-04 - 3s/epoch - 3ms/step
Epoch 5/100
1026/1026 - 3s - loss: 8.2271e-04 - 3s/epoch - 3ms/step
Epoch 6/100
1026/1026 - 2s - loss: 6.5365e-04 - 2s/epoch - 2ms/step
Epoch 7/100
1026/1026 - 3s - loss: 5.8686e-04 - 3s/epoch - 3ms/step
Epoch 8/100
1026/1026 - 3s - loss: 4.9488e-04 - 3s/epoch - 3ms/step
Epoch 9/100
1026/1026 - 3s - loss: 4.8844e-04 - 3s/epoch - 3ms/step
Epoch 10/100
1026/1026 - 3s - loss: 4.7567e-04 - 3s/epoch - 3ms/step
Epoch 11/100
1026/1026 - 4s - loss: 3.6817e-04 - 4s/epoch - 4ms/step
Epoch 12/100
1026/1026 - 3s - loss: 4.1517e-04 - 3s/epoch - 3ms/step
Epoch 13/100
1026/1026 - 4s - loss: 3.5535e-04 - 4s/epoch - 3ms/step
Epoch 14/100
1026/1026 - 3s - loss: 3.4093e-04 - 3s/epoch - 3ms/step
Epoch 15/100
1026/1026 - 3s - loss: 3.1846e-04 - 3s

In [8]:
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

16/16 [==============================] - 0s 2ms/step


In [9]:
train_predict = scaler.inverse_transform(train_predict)
train = scaler.inverse_transform(train)
test_predict = scaler.inverse_transform(test_predict)
test = scaler.inverse_transform(test)

In [10]:
"""train_rmse = np.sqrt(np.mean(np.power((train - train_predict), 2)))
test_rmse = np.sqrt(np.mean(np.power((test - test_predict), 2)))
print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)"""
from sklearn.metrics import r2_score
import math
from sklearn.metrics import mean_squared_error
train_rmse = np.sqrt(mean_squared_error(train, train_predict))
test_rmse = np.sqrt(mean_squared_error(test, test_predict))
print('R2 Score_train:', r2_score(train, train_predict))
print('R2 Score_test:', r2_score(test, test_predict))

print('Train RMSE: %.2f' % train_rmse)
print('Test RMSE: %.2f' % test_rmse)

R2 Score_train: 0.9990911785130775
R2 Score_test: 0.9985882698008293
Train RMSE: 0.91
Test RMSE: 1.12


In [11]:
df=pd.DataFrame({'Actual':train.tolist(),'Predicted from lstm':train_predict.tolist()})
df.to_excel('lstm_10.xlsx')
df

,Actual,Predicted from lstm
0,[112.89999389648438],[111.08251953125]
1,[115.25],[113.33768463134766]
2,[102.4000015258789],[100.94798278808594]
3,[99.75],[98.37677764892578]
4,[83.5199966430664],[82.53852844238281]
...,...,...
1021,[61.599998474121094],[61.00732421875]
1022,[59.52000045776367],[58.9622917175293]
1023,[50.470001220703125],[50.07133102416992]
1024,[47.77000427246094],[47.42237854003906]


In [12]:
df=pd.DataFrame({'Actual':test.tolist(),'Predicted from lstm':test_predict.tolist()})
df.to_excel('lstm_10_test.xlsx')
df

,Actual,Predicted from lstm
0,[79.47000122070312],[78.56782531738281]
1,[80.91999816894531],[79.99005889892578]
2,[92.27999877929688],[91.10442352294922]
3,[92.55000305175781],[91.36785125732422]
4,[110.01000213623047],[108.3022689819336]
...,...,...
501,[73.86000061035156],[73.0599365234375]
502,[81.33999633789062],[80.40189361572266]
503,[96.5999984741211],[95.31428527832031]
504,[52.630001068115234],[52.19192123413086]
